In [223]:
import pandas as pd
import os
import re
import numpy as np
import pprint

In [224]:
df = pd.read_excel("../data/fomc_dissents_data.xlsx",skiprows=3)

In [263]:
df["Date"] = df["FOMC Meeting"].apply(lambda x:str(x).split(" ")[0])
df['FOMC Votes'] = df['FOMC Votes'].apply(lambda x:0 if np.isnan(x) else x)
print(df.columns)

Index(['Year', 'FOMC Meeting', 'Chair', 'Dissent (Y or N)', 'FOMC Votes',
       'Votes for Action', 'Votes Against Action',
       'Number Governors Dissenting', 'Number Presidents Dissenting',
       'No. Governors for Tighter', 'No. Governors for Easier',
       'No. Presidents for Tighter', 'No. Presidents for Easier',
       'Dissenters Tighter', 'Dissenters Easier',
       'Dissenters Other/Indeterminate', 'Date'],
      dtype='object')


In [259]:

voter_df = pd.DataFrame()

for index,row in df.iterrows():
    voters = []
    num_voters = int(row['FOMC Votes'])
    date_path = '../../../collection/python/data/transcript_raw_text/{}.txt'.format(row['Date'])
    if not os.path.exists(date_path):
        continue
    with open(date_path) as f:
        broken = False
        broken_starts = 0
        broken_ends = 0
        lines = f.readlines()
        
        '''First Check For Broken Title'''

        print("CHECKING USING FRAGMENT HEURISTIC")
        for line in lines[:200]:
            if line.strip():
                if broken_ends==0:
                    title_frag = re.match(r'^(?:PRESENT: |PRESENT. )?(?:Mr.|Ms.|Mt. )$',line.strip())
                    if title_frag:
                        if not broken:
                            broken = True
                        print("Broken Begining")
                        print(title_frag.group(0))
                        title_frag_string = str(title_frag.group(0)).replace("PRESENT: ","")
                        voters.append(title_frag_string)
                        broken_starts+=1
                        continue
                if broken and broken_ends<len(voters):
                    name_fragment = re.match('^[A-Z][a-z][A-Za-z]*',line.strip())
                    if name_fragment:
                        print(name_fragment)
                        if broken_ends==0:
                            print("Found {} Fragments".format(len(voters)))
                            print(voters)
                        print("Broken Ending")
                        print(name_fragment.group(0))
                        voters[broken_ends] = voters[broken_ends]+" "+str(name_fragment.group(0))
                        broken_ends+=1
        
        '''Check using Mr. Regex'''
        
        if len(voters)==0:
            print("CHECKING TITLE REGEX")
            for line in lines[:200]:
                '''Then check for valid input'''
                voter_line = re.findall(r'(?:Mr.|Ms.) [A-Z][a-zA-Z]*',line.strip())
                if voter_line:
                    #print(voter_line)
                    voters.append(voter_line[0])
                    if len(voters)>=num_voters:
                        break
        
        '''Check Last Name Regex'''
        
        if len(voters) == 0:
            print("Checking POST-PRESENT-NAME HEURISTIC")
            found_present = False
            for line in lines[:200]:
                if "PRESENT:" in line.strip() or "PRESENT." in line.strip():
                    found_present = True
                    present_line = line.split(",")[0].strip().replace("PRESENT","")
                    name_text = re.match('[A-Z][a-z]*\s?(?:[A-Z][a-z]*)?',present_line)
                    if name_text:
                        voters.append(name_text.group(0))
                    continue
                if found_present:
                    print(line)
                    name_text = re.match('[A-Z][a-z]*\s?(?:[A-Z][a-z]*)?',line.split(",")[0].strip())
                    if name_text:
                        voters.append(name_text.group(0))
                    if len(voters)>=num_voters:
                        break
    print('Date:{}'.format(row['Date']))
    print("Broken Status:{}".format(broken))
    print("Voter Number:{}".format(num_voters))
    print("Voters Found:{}".format(len(voters)))
    pprint.pprint(voters)
    voter_df = voter_df.append({
        "Date":row['FOMC Meeting'],
        "voters_expected":num_voters,
        "voters_observed":len(voters),
        "Voters":voters if num_voters==len(voters) else None,
    },ignore_index=True)
    print("="*50)
print(voter_df)

CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1976-04-20
Broken Status:False
Voter Number:11
Voters Found:11
['Mr. Broida',
 'Mr. Altmann',
 'Mr. Bernard',
 'Mr. O',
 'Mr. Guy',
 'Mr. Axilrod',
 'Mr. Gramley',
 'Mr. Holmes',
 'Mr. Pardee',
 'Mr. Sternlight',
 'Mr. Coyne']
CHECKING USING FRAGMENT HEURISTIC
Broken Begining
PRESENT: Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
<_sre.SRE_Match object; span=(0, 5), match='Burns'>
Found 11 Fragments
['Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.']
Broken Ending
Burns
<_sre.SRE_Match object; span=(0, 7), match='Volcker'>
Broken Ending
Volcker
<_sre.SRE_Match object; span=(0, 6), match='Balles'>
Broken Ending
Balles
<_sre.SRE_Match object; span=(0, 5), match='Black'>
Broken Ending
Black
<_sre.SRE_Match object; span=(0, 8), match='Coldwell'>
Brok

CHECKING USING FRAGMENT HEURISTIC
Broken Begining
PRESENT: Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
<_sre.SRE_Match object; span=(0, 6), match='Miller'>
Found 11 Fragments
['Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.']
Broken Ending
Miller
<_sre.SRE_Match object; span=(0, 7), match='Volcker'>
Broken Ending
Volcker
<_sre.SRE_Match object; span=(0, 8), match='Baughman'>
Broken Ending
Baughman
<_sre.SRE_Match object; span=(0, 8), match='Coldwell'>
Broken Ending
Coldwell
<_sre.SRE_Match object; span=(0, 8), match='Eastburn'>
Broken Ending
Eastburn
<_sre.SRE_Match object; span=(0, 7), match='Gardner'>
Broken Ending
Gardner
<_sre.SRE_Match object; span=(0, 7), match='Jackson'>
Broken Ending
Jackson
<_sre.SRE_Match object; span=(0, 6), match='Partee'>
Broken Ending
Partee
<_sre.SRE_Match object; span=(

CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1981-03-31
Broken Status:False
Voter Number:11
Voters Found:11
['Mr. Volcker',
 'Mr. Solomon',
 'Mr. Boehne',
 'Mr. Boykin',
 'Mr. Corrigan',
 'Mr. Partee',
 'Mr. Rice',
 'Mr. Schultz',
 'Mr. Wallich',
 'Mr. Axilrod',
 'Mr. Altmann']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1981-05-06
Broken Status:False
Voter Number:9
Voters Found:9
['Mr. Volcker',
 'Mr. Solomon',
 'Mr. Boehne',
 'Mr. Boykin',
 'Mr. Corrigan',
 'Mr. Gramley',
 'Mr. Rice',
 'Mr. Schultz',
 'Mr. Axilrod']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1981-05-18
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Volcker',
 'Mr. Solomon',
 'Mr. Boehne',
 'Mr. Boykin',
 'Mr. Corrigan',
 'Mr. Gramley',
 'Mr. Partee',
 'Mr. Rice',
 'Mr. Schultz',
 'Mr. Wallich',
 'Mr. Axilrod',
 'Mr. Altmann']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1981-08-18
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Volcke

Broken Begining
Ms.
Broken Begining
Mr.
<_sre.SRE_Match object; span=(0, 9), match='Greenspan'>
Found 11 Fragments
['Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Ms.', 'Mr.']
Broken Ending
Greenspan
<_sre.SRE_Match object; span=(0, 8), match='Corrigan'>
Broken Ending
Corrigan
<_sre.SRE_Match object; span=(0, 6), match='Angell'>
Broken Ending
Angell
<_sre.SRE_Match object; span=(0, 6), match='Boehne'>
Broken Ending
Boehne
<_sre.SRE_Match object; span=(0, 6), match='Boykin'>
Broken Ending
Boykin
<_sre.SRE_Match object; span=(0, 6), match='Heller'>
Broken Ending
Heller
<_sre.SRE_Match object; span=(0, 7), match='Johnson'>
Broken Ending
Johnson
<_sre.SRE_Match object; span=(0, 5), match='Keehn'>
Broken Ending
Keehn
<_sre.SRE_Match object; span=(0, 6), match='Kelley'>
Broken Ending
Kelley
<_sre.SRE_Match object; span=(0, 5), match='Seger'>
Broken Ending
Seger
<_sre.SRE_Match object; span=(0, 5), match='Stern'>
Broken Ending
Stern
Date:1987-11-03
Broken Status:True
Voter Nu

CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1991-11-05
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Greenspan',
 'Mr. Corrigan',
 'Mr. Angell',
 'Mr. Black',
 'Mr. Forrestal',
 'Mr. Keehn',
 'Mr. Kelley',
 'Mr. LaWare',
 'Mr. Mullins',
 'Mr. Parry']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1991-12-17
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Greenspan',
 'Mr. Corrigan',
 'Mr. Angell',
 'Mr. Black',
 'Mr. Forrestal',
 'Mr. Keehn',
 'Mr. Kelley',
 'Mr. LaWare',
 'Mr. Lindsey',
 'Mr. Mullins',
 'Mr. Parry',
 'Ms. Phillips']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1992-03-31
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Greenspan',
 'Mr. Corrigan',
 'Mr. Angell',
 'Mr. Hoenig',
 'Mr. Jordan',
 'Mr. Kelley',
 'Mr. LaWare',
 'Mr. Lindsey',
 'Mr. Melzer',
 'Mr. Mullins',
 'Ms. Phillips',
 'Mr. Syron']
CHECKING USING FRAGMENT HEURISTIC
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Brok

Date:1996-09-24
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Boehne',
 'Mr. Jordan',
 'Mr. Kelley',
 'Mr. Lindsey',
 'Mr. McTeer',
 'Ms. Phillips',
 'Ms. Rivlin',
 'Mr. Stern',
 'Ms. Yellen',
 'Ms. Minehan']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1996-11-13
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Boehne',
 'Mr. Jordan',
 'Mr. Kelley',
 'Mr. Lindsey',
 'Mr. McTeer',
 'Mr. Meyer',
 'Ms. Phillips',
 'Ms. Rivlin',
 'Mr. Stern',
 'Ms. Yellen']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1996-12-17
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Boehne',
 'Mr. Jordan',
 'Mr. Kelley',
 'Mr. Lindsey',
 'Mr. McTeer',
 'Mr. Meyer',
 'Ms. Phillips',
 'Ms. Rivlin',
 'Mr. Stern',
 'Ms. Yellen']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1997-03-25
Broken Status:False
Voter Number:10
Voters Found:10


CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2001-11-06
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Ferguson',
 'Mr. Gramlich',
 'Mr. Hoenig',
 'Mr. Kelley',
 'Mr. Meyer',
 'Ms. Minehan',
 'Mr. Moskow',
 'Mr. Poole']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2001-12-11
Broken Status:False
Voter Number:11
Voters Found:11
['Mr. Greenspan',
 'Mr. McDonough',
 'Ms. Bies',
 'Mr. Ferguson',
 'Mr. Gramlich',
 'Mr. Hoenig',
 'Mr. Meyer',
 'Ms. Minehan',
 'Mr. Moskow',
 'Mr. Olson',
 'Mr. Poole']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2002-03-19
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Greenspan',
 'Mr. McDonough',
 'Ms. Bies',
 'Mr. Ferguson',
 'Mr. Gramlich',
 'Mr. Jordan',
 'Mr. McTeer',
 'Mr. Olson',
 'Mr. Santomero',
 'Mr. Stern']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2002-05-07
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Greenspan',
 

CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2007-09-18
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Bernanke',
 'Mr. Geithner',
 'Mr. Evans',
 'Mr. Hoenig',
 'Mr. Kohn',
 'Mr. Kroszner',
 'Mr. Mishkin',
 'Mr. Poole',
 'Mr. Rosengren',
 'Mr. Warsh']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2007-12-11
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Bernanke',
 'Mr. Geithner',
 'Mr. Evans',
 'Mr. Hoenig',
 'Mr. Kohn',
 'Mr. Kroszner',
 'Mr. Mishkin',
 'Mr. Poole',
 'Mr. Rosengren',
 'Mr. Warsh']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2008-01-21
Broken Status:False
Voter Number:9
Voters Found:9
['Mr. Bernanke',
 'Mr. Geithner',
 'Mr. Evans',
 'Mr. Hoenig',
 'Mr. Kohn',
 'Mr. Kroszner',
 'Mr. Poole',
 'Mr. Rosengren',
 'Mr. Warsh']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2008-03-18
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Bernanke',
 'Mr. Geithner',
 'Mr. Fisher',
 'Mr. Kohn',

In [260]:
#print(len(voter_df[voter_df["Voters"].isna()]))
voter_df[voter_df["Voters"].isna()].reset_index()

,index,Date,Voters,voters_expected,voters_observed
0,21,1978-05-05,None,10.0,4.0
1,31,1979-03-20,None,10.0,8.0
2,36,1979-09-18,None,12.0,6.0
3,37,1979-10-06,None,12.0,9.0
4,39,1980-03-07,None,11.0,5.0
5,43,1980-05-20,None,11.0,7.0
6,44,1980-07-09,None,12.0,7.0
7,46,1980-09-16,None,12.0,6.0
8,48,1980-11-18,None,12.0,7.0
9,49,1980-11-26,None,11.0,5.0


In [261]:
voter_df[voter_df.voters_observed==0]

,Date,Voters,voters_expected,voters_observed
207,2010-03-16,None,10.0,0.0
208,2010-08-10,None,10.0,0.0
209,2010-09-21,None,9.0,0.0
210,2010-12-14,None,11.0,0.0
211,2011-03-15,None,10.0,0.0
212,2011-08-09,None,10.0,0.0
213,2011-12-13,None,10.0,0.0
